<a href="https://colab.research.google.com/github/PabloPalaciosDev/BackendEjemplo/blob/main/chatbotWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importamos librerias**

In [ ]:
import json
import nltk
import tensorflow as tf
import numpy as np
import re

Cargamos el archivo json con estos metodos para volverlo un diccionario de python manipulable

In [ ]:
with open("data.json") as json_file:
    data = json.load(json_file)
print(data)

{'data': [{'question': '¿Por qué debería contratar sus servicios?', 'answer': 'Contamos con más de 10 años de experiencia entregando soluciones tecnológicas únicas en todas las industrias y áreas de desarrollo. Nos enfocamos especialmente en generar resultados asombrosos en 10 áreas principales de desarrollo pero podemos brindar experiencias personalizadas  sin importar sus requisitos tecnológicos.', 'context': ['Info de Paquetes', 'Experiencia', 'Contrato'], 'synonyms': ['Contar', 'Paquetes'], 'related_question': ['¿Cuáles son sus áreas de conocimiento?', '¿Qué les hace diferente a los demás?']}, {'question': '¿Cuáles son sus áreas de conocimiento?', 'answer': 'Entre las áreas tecnológicas que podemos trabajar, podemos enumerar: Data Science y aprendizaje automático, pruebas unitarias, desarrollo de plataformas, microservicios, diseño de producto, desarrollo frontend, estrategia de producto e internet de las cosas. ', 'context': ['Contrato', 'Experiencia'], 'synonyms': ['Enfoque', 'Ru

Indivualmente, guardamos preguntas y respuestas en listas separadas para el preprocesamiento

In [ ]:
questions = []
for obj in data['data']:
    question = obj['question']
    questions.append(question)

answers = []
for obj in data['data']:
    answer = obj['answer']
    answers.append(answer)

synonyms= []
for obj in data['data']:
    for synonym in obj['synonyms']:
        synonyms.append(synonym)


contexts = []
for obj in data['data']:
  for context in obj['context']:
    contexts.append(context)


related_questions = []
for obj in data['data']:
  for related_question in obj['related_question']:
    related_questions.append(related_question)
    
print(questions)
print(answers)
print(synonyms)
print(contexts)
print(related_questions)


['¿Por qué debería contratar sus servicios?', '¿Cuáles son sus áreas de conocimiento?', '¿Existen algunas empresas que han contado con sus servicios?', '¿Trabajan para un tipo de industria en específico?', '¿Qué les hace diferente a los demás?', '¿Dónde están ubicados?', '¿Ofrecen servicios de alojamiento web?', 'Necesito desarrollar un sitio web', 'Busco un software que ayude a mi empresa a realizar una tarea en específico.', '¿Tienen servicios de mercadeo por correo electrónico?', '¿Qué tecnologías manejan al momento de trabajar una página web?', '¿Cuánto tiempo tomaría hacer mi página web?', 'Si luego de tener mi página web ya creada, surge algún problema, ¿Podrían ayudarme con ello?', 'Quiero tener un dominio propio.', '¿Cómo es el proceso de mi diseño web?', '¿Tambien hacen logos para mi página web?', '¿Cuánto tiempo demoran haciendo mi página web después de haberles entregado toda mi info?', '¿Mi página web puede tener un sistema de reservas?', '¿Qué funciones pueden agregarles a

COMIENZA EL PREPROCESAMIENTO DE LOS DATOS

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords_list = set(stopwords.words('spanish'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**ELIMINACIÓN DE STOPWORDS**

In [ ]:
#Aqui pasamos las listas por una funcion que elimina stopwords(palabras que no otorgan sentido a la pregunta o respuesta)
def eliminar_stopwords(cadenas):
    cadenas_sin_stopwords = []
    for cadena in cadenas:
        palabras = cadena.split()
        palabras_filtradas = [palabra for palabra in palabras if palabra.lower() not in stopwords_list]
        cadena_sin_stopwords = " ".join(palabras_filtradas)
        cadenas_sin_stopwords.append(cadena_sin_stopwords)
    return cadenas_sin_stopwords

#lo aplicamos a las listas necesarias, context y sinonimos quedan fuera porque son palabras
questions_sin_stopwords = eliminar_stopwords(questions)
answer_sin_stopwords = eliminar_stopwords(answers)
related_questions_sin_stopwords = eliminar_stopwords(related_questions)


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

TOKENIZAMOS Y ELIMINAMOS SIGNOS DE PUNTUACION

In [ ]:
def tokenizar(datos_de_conversacion):
  listas_tokenizadas = []
  for i in range(len(datos_de_conversacion)):
    datos_de_conversacion[i] = re.sub(r'[^\w\s]','',datos_de_conversacion[i].lower())
  
  for token in datos_de_conversacion:
    tokens = word_tokenize(token)
    listas_tokenizadas.append(tokens)

  return listas_tokenizadas

In [ ]:
questions_tokenizadas = tokenizar(questions_sin_stopwords)
answer_tokenizadas = tokenizar(answer_sin_stopwords)
related_questions_tokenizadas = tokenizar(related_questions_sin_stopwords)

In [ ]:
!python -m spacy download es_core_news_sm

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-18 16:14:02.440147: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 12.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
import spacy

nlp = spacy.load("es_core_news_sm")

def lemmatize_list(words_list):
    lemmatized_words = []
    for words in words_list:
        lemmatized_words.append([token.lemma_ for token in nlp(" ".join(words))])
    return lemmatized_words


In [ ]:
questions_lematizadas = lemmatize_list(questions_tokenizadas )

In [ ]:
print(questions_lematizadas)

[['por', 'deber', 'contratar', 'servicio'], ['cua él', 'área', 'conocimiento'], ['existir', 'empresa', 'contado', 'servicio'], ['trabajar', 'tipo', 'industria', 'específico'], ['qué', 'hacer', 'diferente', 'demá'], ['dónde', 'ubicado'], ['ofrecer', 'servicio', 'alojamiento', 'web'], ['necesitar', 'desarrollar', 'sitio', 'web'], ['buscar', 'software', 'ayudar', 'empresa', 'realizar', 'tarea', 'específico'], ['tener', 'servicio', 'mercadeo', 'correo', 'electrónico'], ['qué', 'tecnología', 'manejar', 'momento', 'trabajar', 'página', 'web'], ['cuánto', 'tiempo', 'tomar', 'hacer', 'página', 'web'], ['si', 'luego', 'tener', 'página', 'web', 'creado', 'surgir', 'alguno', 'problema', 'poder', 'ayudar yo', 'él'], ['querer', 'tener', 'dominio', 'propio'], ['cómo', 'proceso', 'diseño', 'web'], ['tambien', 'hacer', 'logo', 'página', 'web'], ['cuánto', 'tiempo', 'demorar', 'hacer', 'página', 'web', 'después', 'haberl', 'entregar', 'todo', 'info'], ['mi', 'página', 'web', 'poder', 'tener', 'sistema'